In [1]:
import torch; 
print(torch.cuda.is_available());

True


In [2]:
import torch

# Check if CUDA (NVIDIA GPU) is available
if torch.cuda.is_available():
    print("GPU is available.")
    print("GPU name: ", torch.cuda.get_device_name(0))
else:
    print("No GPU found.")

GPU is available.
GPU name:  NVIDIA GeForce RTX 2070


In [5]:
!pip install -U ultralytics
!pip install supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.7/877.7 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.23
    Uninstalling ultralytics-8.3.23:
      Successfully uninstalled ultralytics-8.3.23


In [3]:

import shutil
from pathlib import Path
import sys

from matplotlib import pyplot as plt
import skimage.io as io

import torch
from torch import cuda
import supervision as sv
from ultralytics import YOLO

import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
print(sys.version)

has_gpu = cuda.is_available()
device = torch.device('cuda' if has_gpu else 'cpu')

print(device)
if has_gpu:
    print(cuda.get_device_name(0))
    

3.10.15 | packaged by conda-forge | (main, Oct 16 2024, 01:24:24) [GCC 13.3.0]
cuda
NVIDIA GeForce RTX 2070


Creating path variables for our new project 

In [12]:
path_map = {'ROOT': Path('craterdetectionluna')}
path_map['DS'] = path_map['ROOT'] / 'datasets'

# We will treat DS_CRATERS as the repository directory containing the
# original images and labels that are provided with the dataset. This helps
# keep things clean if we want to make another dataset using the same data
# (which we'll do in a few steps)
path_map['DS_CRATERS'] = path_map['DS'] / 'craters'
path_map['DS_CRATERS_IMAGES'] = path_map['DS_CRATERS'] / 'images'
path_map['DS_CRATERS_LABELS'] = path_map['DS_CRATERS'] / 'labels'

# Here is our project paths. We'll also keep this in DS for simplicity.
# We need three directories for our dataset: root, images, labels.
# In this case, "root" will be the project name. Additionally, we will define
# the path and name of another file we'll make later on.
path_map['PROJECT_NAME'] = "yolov9c_craters"
path_map['DS_PROJECT'] = path_map['DS'] / path_map['PROJECT_NAME']
path_map['PROJECT_IMAGES'] = path_map['DS_PROJECT'] / 'images'
path_map['PROJECT_LABELS'] = path_map['DS_PROJECT'] / 'labels'
path_map['PROJECT_DATA_YAML'] = path_map['DS_PROJECT'] / 'data.yaml'

# Declare the output paths that are created by the model automatically by
# the model. We'll declare them now because we will use them later.
path_map['RUNS'] = path_map['ROOT'] / 'runs'
path_map['DETECT'] = path_map['RUNS'] / 'detect'
path_map['PROJECT_RESULTS'] = path_map['DETECT'] / path_map['PROJECT_NAME']

In [13]:
def create_project_directories():
  print("Creating directory structure")
  for name, path in path_map.items():
      if isinstance(path, Path) and not path.suffix:
          path = path.resolve()
          path_map[name] = path
          path.mkdir(parents=True, exist_ok=True)
          print('  ',path)
  print("Complete")

create_project_directories()

Creating directory structure
   /home/microuasml/tmp/craterdetectionluna
   /home/microuasml/tmp/craterdetectionluna/datasets
   /home/microuasml/tmp/craterdetectionluna/datasets/craters
   /home/microuasml/tmp/craterdetectionluna/datasets/craters/images
   /home/microuasml/tmp/craterdetectionluna/datasets/craters/labels
   /home/microuasml/tmp/craterdetectionluna/datasets/yolov9c_craters
   /home/microuasml/tmp/craterdetectionluna/datasets/yolov9c_craters/images
   /home/microuasml/tmp/craterdetectionluna/datasets/yolov9c_craters/labels
   /home/microuasml/tmp/craterdetectionluna/runs
   /home/microuasml/tmp/craterdetectionluna/runs/detect
   /home/microuasml/tmp/craterdetectionluna/runs/detect/yolov9c_craters
Complete


In [15]:
from PIL import Image
import os

# Path to your images directory
images_dir = path_map['ROOT'] / "rocksdataset/images/render"

# Loop through each image in the directory to find the dimensions
for image_file in os.listdir(images_dir):
    if image_file.endswith((".tif", ".jpg", ".png")):  # Add any other extensions as needed
        image_path = os.path.join(images_dir, image_file)
        
        # Open the image
        with Image.open(image_path) as img:
            width, height = img.size
            print(f"Image: {image_file}, Width: {width}, Height: {height}")
        
        # Uncomment the following line if all images are the same size and you just need one sample
        break

Image: render3064.png, Width: 720, Height: 480


In [18]:
import pandas as pd
import os

# Load the CSV file
csv_path = path_map['ROOT'] / "rocksdataset/bounding_boxes.csv"
output_dir = path_map['ROOT'] / "labels"
image_width = 720  # Replace with the actual image width
image_height = 480  # Replace with the actual image height

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Read the CSV
df = pd.read_csv(csv_path)

# Process each row in the CSV
for _, row in df.iterrows():
    # Extract data from the CSV
    frame_number = int(row["Frame"])
    top_left_x = row["TopLeftCornerX"]
    top_left_y = row["TopLeftCornerY"]
    length = row["Length"]
    height = row["Height"]
    class_id = 0  # Assuming a single class (e.g., rock)

    # Convert to YOLO format
    center_x = (top_left_x + length / 2) / image_width
    center_y = (top_left_y + height / 2) / image_height
    width = length / image_width
    height = height / image_height

    # Construct the output filename
    output_filename = f"render{frame_number:04d}.txt"
    output_path = os.path.join(output_dir, output_filename)

    # Write the bounding box to the file
    with open(output_path, "a") as f:
        f.write(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}\n")

print("Conversion complete. YOLO annotation files saved in 'labels' directory.")


Conversion complete. YOLO annotation files saved in 'labels' directory.


In [37]:
import yaml

def make_data_yaml(data_yaml_path, abs_root_path, rel_train_images_path, rel_val_images_path, class_map):
    """
    Creates a YAML configuration file for YOLO with plain text formatting.

    Parameters:
    - data_yaml_path (str): Path where the YAML file will be saved.
    - abs_root_path (str): Absolute root path of the dataset.
    - rel_train_images_path (str): Relative path to the training images directory from the root.
    - rel_val_images_path (str): Relative path to the validation images directory from the root.
    - class_map (dict): Dictionary mapping class IDs to class names.
    """
    with open(data_yaml_path, 'w') as f:
        f.write(f"path: {abs_root_path}\n")
        f.write(f"train: {rel_train_images_path}\n")  # Use relative path for images
        f.write(f"val: {rel_val_images_path}\n")
        f.write("names:\n")
        for id, name in class_map.items():
            f.write(f"  {id}: {name}\n")
    
    print(f"Created '{data_yaml_path}' with paths as plain strings.")


In [22]:
import os
import shutil
import random

def split_dataset(images_dir, labels_dir, output_dir, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    # Check if ratios sum to 1.0
    assert train_ratio + val_ratio + test_ratio == 1.0, "Ratios must sum to 1.0"

    # Paths for train, val, and test
    for subset in ["train", "val", "test"]:
        os.makedirs(os.path.join(output_dir, "images", subset), exist_ok=True)
        os.makedirs(os.path.join(output_dir, "labels", subset), exist_ok=True)

    # Get list of all images and shuffle
    images = [img for img in os.listdir(images_dir) if img.endswith((".jpg", ".png", ".tif"))]
    random.shuffle(images)

    # Split images into train, val, and test
    train_count = int(len(images) * train_ratio)
    val_count = int(len(images) * val_ratio)

    splits = {
        "train": images[:train_count],
        "val": images[train_count:train_count + val_count],
        "test": images[train_count + val_count:]
    }

    # Move images and labels to respective folders
    for subset, files in splits.items():
        for image in files:
            image_path = os.path.join(images_dir, image)
            label_path = os.path.join(labels_dir, image.replace(".jpg", ".txt").replace(".png", ".txt").replace(".tif", ".txt"))

            if os.path.exists(label_path):
                shutil.copy(image_path, os.path.join(output_dir, "images", subset, image))
                shutil.copy(label_path, os.path.join(output_dir, "labels", subset, os.path.basename(label_path)))
            else:
                print(f"Warning: Label not found for {image}")

    print("Dataset split complete.")

In [26]:
split_dataset(
    images_dir=path_map['ROOT'] / "rocksdataset/images/render",
    labels_dir=path_map['ROOT'] /"labels",
    output_dir= path_map['PROJECT_IMAGES'],
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1
)

Dataset split complete.


In [45]:
# Adjusted usage example with the new function parameters
make_data_yaml(
    data_yaml_path="dataset.yaml",
    abs_root_path=str(path_map['ROOT'] / "datasets/yolov9c_craters"),  # Absolute root path to dataset
    rel_train_images_path="images/images/train",               # Relative path from root
    rel_val_images_path="images/images/val",                   # Relative path from root
    class_map={0: "rock"}                                      # Class map with ID and class name
)


Created 'dataset.yaml' with paths as plain strings.


In [ ]:
from ultralytics import YOLO

# Initialize the model with pretrained weights
model = YOLO("yolov9c.pt")

model.train(
    data="dataset.yaml",        # Path to dataset YAML
    epochs=300,                 # Number of training epochs
    patience=25,                # Early stopping patience
    imgsz=512,                  # Image size
    batch=16,                   # Batch size
    iou=0.5,                    # IoU threshold for validation
    freeze=18                   # Freeze the first 18 layers
)


New https://pypi.org/project/ultralytics/8.3.26 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.24 🚀 Python-3.10.15 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce RTX 2070, 7952MiB)
engine/trainer: task=detect, mode=train, model=yolov9c.pt, data=dataset.yaml, epochs=300, time=None, patience=25, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train15, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=18, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=

train: Scanning /home/microuasml/tmp/craterdetectionluna/datasets/yolov9c_craters/images/labels/train.cache... 4979 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4979/4979 [00:00<?, ?it/s]
val: Scanning /home/microuasml/tmp/craterdetectionluna/datasets/yolov9c_craters/images/labels/val.cache... 616 images, 0 backgrounds, 0 corrupt: 100%|██████████| 616/616 [00:00<?, ?it/s]


Plotting labels to runs/detect/train15/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 154 weight(decay=0.0), 161 weight(decay=0.0005), 160 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 512 train, 512 val
Using 8 dataloader workers
Logging results to runs/detect/train15
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/300      1.65G      1.754      1.971      1.462         22        512: 100%|██████████| 312/312 [00:54<00:00,  5.70it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:04<00:00,  4.78it/s]

                   all        616       1921      0.597      0.504      0.519      0.271


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     80/300      1.63G      1.443      1.158      1.137         13        512: 100%|██████████| 312/312 [00:51<00:00,  6.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:04<00:00,  4.99it/s]

                   all        616       1921      0.756      0.636      0.698      0.411



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     81/300       1.9G      1.435      1.143      1.141         50        512:  71%|███████   | 222/312 [00:37<00:15,  5.71it/s]